In [4]:
# !unzip ./data.zip

In [296]:
!nvidia-smi

Thu Apr 15 23:31:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 00000000:26:00.0  On |                  N/A |
|  0%   57C    P5    15W / 170W |    450MiB / 12045MiB |     16%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [297]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.naive_bayes import GaussianNB
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler

In [298]:
test_data = pd.read_csv('./test.csv')
data = pd.read_csv('./train.csv')

In [299]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [300]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [301]:
len(data)

891

In [302]:
data.drop('Cabin',axis=1,inplace=True)

In [303]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [304]:
data['Age'].fillna(data['Age'].mean(),inplace=True)

In [305]:
data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

In [306]:
data.dropna(inplace=True)

In [307]:
data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [308]:
X = data.drop('Survived',axis=1)
y = data['Survived']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125)
len(X_train),len(X_test)

(777, 112)

In [309]:
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
612,613,3,"Murphy, Miss. Margaret Jane",female,29.699118,1,0,367230,15.5000,Q
130,131,3,"Drazenoic, Mr. Jozef",male,33.000000,0,0,349241,7.8958,C
784,785,3,"Ali, Mr. William",male,25.000000,0,0,SOTON/O.Q. 3101312,7.0500,S
479,480,3,"Hirvonen, Miss. Hildur E",female,2.000000,0,1,3101298,12.2875,S
756,757,3,"Carlsson, Mr. August Sigfrid",male,28.000000,0,0,350042,7.7958,S
...,...,...,...,...,...,...,...,...,...,...
82,83,3,"McDermott, Miss. Brigdet Delia",female,29.699118,0,0,330932,7.7875,Q
600,601,2,"Jacobsohn, Mrs. Sidney Samuel (Amy Frances Chr...",female,24.000000,2,1,243847,27.0000,S
701,702,1,"Silverthorne, Mr. Spencer Victor",male,35.000000,0,0,PC 17475,26.2875,S
663,664,3,"Coleff, Mr. Peju",male,36.000000,0,0,349210,7.4958,S


In [310]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Embarked        object
dtype: object

In [311]:
ct = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'),['Name','Sex','Parch','Embarked']),
    (MinMaxScaler(),['PassengerId','Pclass','Age','SibSp','Parch','Fare'])
)
ct.fit(X)
X_train = ct.transform(X_train)
X_test = ct.transform(X_test)

/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [312]:
X_train = X_train.toarray()
X_test = X_test.toarray()
# X_train = np.array(X_train).reshape(1,-1)
# X_test = np.array(X_test).reshape(1,-1)
# y_train = np.array(y_train).reshape(1,-1)
# y_test = np.array(y_test).reshape(1,-1)

# Modelling

In [355]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.svm import SVC,SVR
from sklearn.linear_model import SGDClassifier,SGDRegressor
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn.naive_bayes import GaussianNB
models = [
          ['RandomForestClassifier',RandomForestClassifier],
          ['DecisionTreeClassifier',DecisionTreeClassifier],
          ['SVC',SVC],
          ['SGDClassifier',SGDClassifier],
          ['KNeighborsClassifier',KNeighborsClassifier],
          ['GaussianNB',GaussianNB],
          ['RandomForestRegressor',RandomForestRegressor],
          ['DecisionTreeRegressor',DecisionTreeRegressor],
          ['SVR',SVR],
          ['SGDRegressor',SGDRegressor],
          ['KNeighborsRegressor',KNeighborsRegressor],
]
results = {}
for model in models:
  model_name = model[0]
  model = model[1]()
  model.fit(X_train,y_train)
  results[model_name] = model.score(X_test,y_test)
  print(f'{model_name} {model.score(X_test,y_test)}')

/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


RandomForestClassifier 0.8392857142857143
DecisionTreeClassifier 0.8392857142857143
SVC 0.5714285714285714
SGDClassifier 0.8303571428571429
KNeighborsClassifier 0.8392857142857143
GaussianNB 0.4375


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor 0.5424479166666665
DecisionTreeRegressor 0.27083333333333315


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR 0.3903638915314067
SGDRegressor 0.4352623526252546
KNeighborsRegressor 0.5260416666666665


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [358]:
results

{'RandomForestClassifier': 0.8392857142857143,
 'DecisionTreeClassifier': 0.8392857142857143,
 'SVC': 0.5714285714285714,
 'SGDClassifier': 0.8303571428571429,
 'KNeighborsClassifier': 0.8392857142857143,
 'GaussianNB': 0.4375,
 'RandomForestRegressor': 0.5424479166666665,
 'DecisionTreeRegressor': 0.27083333333333315,
 'SVR': 0.3903638915314067,
 'SGDRegressor': 0.4352623526252546,
 'KNeighborsRegressor': 0.5260416666666665}

In [314]:
# grid = {
#     "n_estimators":[100,1125,1250,2500],
#     "max_depth":[None,5,10,12],
#     "max_features":["auto","sqrt",'log2'],
#     "min_samples_split":[round(2.5),5],
#     "min_samples_leaf":[round(2.5),5],
#     'bootstrap':[False,True],
#     'warm_start':[False,True]
# }
# model = RandomForestClassifier()
# model = GridSearchCV(model,param_grid=grid,cv=12,verbose=50)
# model.fit(X_train,y_train)

In [315]:
model = RandomForestClassifier(n_estimators='warn',max_depth=None,max_features='auto',min_samples_split=2,min_samples_leaf=1,warm_start=False,bootstrap=True)
model.fit(X_train,y_train)

/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [316]:
# GridSearchCV(cv=12, error_score='raise-deprecating',
#        estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#             max_depth=None, max_features='auto', max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=1, min_samples_split=2,
#             min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
#             oob_score=False, random_state=None, verbose=0,
#             warm_start=False),
#        fit_params=None, iid='warn', n_jobs=None,
#        param_grid={'n_estimators': [100, 1125, 1250, 2500], 'max_depth': [None, 5, 10, 12], 'max_features': ['auto', 'sqrt', 'log2'], 'min_samples_split': [2, 5], 'min_samples_leaf': [2, 5], 'bootstrap': [False, True], 'warm_start': [False, True]},
#        pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
#        scoring=None, verbose=50)

In [317]:
import pickle
pickle.dump(model,open('./model.sav','wb'))
model_ = pickle.load(open('./model.sav','rb'))

In [318]:
print(f'Score Train : {model.score(X_train,y_train)}')
print(f'Score Test : {model.score(X_test,y_test)}')

Score Train : 0.9806949806949807
Score Test : 0.8482142857142857


In [319]:
print('Done')

Done


In [320]:
X

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C
2,3,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,S
4,5,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,S
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,S
889,890,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C


In [321]:
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [322]:
test_data = test_data.drop('Cabin',axis=1)

In [323]:
X

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C
2,3,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,S
4,5,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,S
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,S
889,890,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C


In [324]:
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,S


In [325]:
test_data['Age'].fillna(test_data['Age'].mean(),inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace=True)

In [326]:
test_data.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [327]:
test_data_old = test_data

In [328]:
# ct = make_column_transformer(
#     (OneHotEncoder(),['Name','Sex','Parch','Embarked']),
#     (MinMaxScaler(),['PassengerId','Pclass','Age','SibSp','Parch','Fare'])
# )
# ct.fit(test_data)
test_data = ct.transform(test_data)
test_data = test_data.toarray()

In [336]:
preds = model.predict(test_data)

In [337]:
test_sub = pd.read_csv('./gender_submission.csv')

In [338]:
test_sub

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [339]:
ids = test_data_old['PassengerId']

In [350]:
submission = pd.DataFrame({'PassengerId':ids,'Survived':preds})

In [351]:
submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [353]:
submission.to_csv('submission.csv',index=False)